In [149]:
import requests
import csv
import re
import datetime
import time
import os
import pandas as pd

# 配置
TEST_COUNT="03"

META_DATA_FILE = 'data/bench_test{}/stress.log'.format(TEST_COUNT)
META_DATA_OUT_FILE = 'data/bench_data{}/meta.csv'.format(TEST_COUNT)
PROMETHEUS_URL = 'http://10.208.130.243:9090'
DATA_STEP = 2
QUERYS = {
    "host_cpu_usage_total": ('range_series','clamp_min(1 - avg(rate(node_cpu_seconds_total{cpu="131",mode="idle"}[1m])),0)'),
    "host_cpu_usage_user": ('range_series','avg by(mode)(rate(node_cpu_seconds_total{cpu="131",mode="user"}[1m]))'),
    "host_cpu_usage_system": ('range_series','avg by(mode)(rate(node_cpu_seconds_total{cpu="131",mode="system"}[1m]))'),
    "host_cpu_usage_iowait": ('range_series','avg by(mode)(rate(node_cpu_seconds_total{cpu="131",mode="iowait"}[1m]))'),
    "host_mem_usage_total": ('range_series','node_memory_MemTotal_bytes - node_memory_MemFree_bytes'),
    "host_mem_bandwith_occupied": ('range_series','rate(resctrl_mem_bandwidth_local_bytes{group="global",numa="2"}[1m])'),
    "host_context_switche": ('range_series','rate(node_context_switches_total{instance="worknode:9100", job="node_exporter"}[1m])'),
    "host_intr": ('range_series','rate(node_intr_total{}[1m])'),
    "host_load1": ('range_series','node_load1 / on (instance) (count by (instance) (node_cpu_seconds_total{mode="idle"}))'),

    "vm_cpu_usage_system": ('range_series','clamp_min(rate(libvirt_domain_cpu_time_sys{domain="podman-alpine317_default", instance="worknode:9177", job="libvirt_exporter"}[1m]), 0) / 1000'),
    "vm_cpu_usage_user": ('range_series','clamp_min(rate(libvirt_domain_cpu_time_user{domain="podman-alpine317_default", instance="worknode:9177", job="libvirt_exporter"}[1m]), 0) / 1000'),
    "vm_cpu_usage_total": ('range_series','sum(rate(libvirt_domain_vcpu_time_total{domain="podman-alpine317_default", instance="worknode:9177", job="libvirt_exporter"}[1m])) by(domain) / 1000'),
    "vm_mem_usage": ('range_series','libvirt_domain_memory_stats_used_percent'),
    "vm_mem_bandwith_occupied": ('range_series','rate(resctrl_mem_bandwidth_local_bytes{group!="global",numa="2"}[1m])'),
    "vm_cpu_llc_occupied": ('range_series', 'sum( resctrl_llc_occupancy_bytes{group!="global"} / on(numa) group_left resctrl_llc_occupancy_bytes{group="global"})'),
    "vm_mkpi": ('range_series','1000 * clamp_min(idelta(libvirt_domain_perf_count{domain="podman-alpine317_default",instance="worknode:9177", job="libvirt_exporter",event="cache_misses"}[1m]), 0)/on(domain) clamp_min(idelta(libvirt_domain_perf_count{domain="podman-alpine317_default", instance="worknode:9177", job="libvirt_exporter", event="instructions"}[1m]), 0)'),
    "vm_network_receive": ('range_series','sum(rate(libvirt_domain_interface_stats_receive_bytes_total{domain="podman-alpine317_default", instance="worknode:9177", job="libvirt_exporter"}[1m])) by(domain, instance) / 1024'),
    "vm_network_transmit": ('range_series','sum(rate(libvirt_domain_interface_stats_transmit_bytes_total{domain="podman-alpine317_default", instance="worknode:9177", job="libvirt_exporter"}[1m])) by(domain, instance) / 1024'),

    "application_p99": ('point_bucket','envoy_redis_command_set_latency_bucket + envoy_redis_command_get_latency_bucket - (envoy_redis_command_set_latency_bucket offset {}s + envoy_redis_command_get_latency_bucket offset {}s)'),
    "application_qps": ('range_series','rate(envoy_redis_command_get_total[1m]) + rate(envoy_redis_command_set_total[1m])'),
}




def fetch_data_save_to_csv():
    with open(META_DATA_OUT_FILE, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            workload_start = row['workload_start']
            workload_end = row['workload_end']
            bench_params = row['params']
            exec = row['exec']

            
            start =  float(workload_start)
            end = float(workload_end)

            series_cdf = []
            bucket_cdf = []
            for metric_name,(kind, query) in QUERYS.items():
                # 查询 prometheus
                if kind == "range_series":
                    params = {
                        'query': query,
                        'start': start,
                        'end': end,
                        'step': DATA_STEP
                    }
                    response = requests.get(f'{PROMETHEUS_URL}/api/v1/query_range', params=params)
                    response.raise_for_status()  # Raise exception if invalid response
                    data = response.json()['data']['result']
                    
                    for metric_data in data:
                        times, values = zip(*metric_data['values'])
                        df = pd.DataFrame({
                            'timestamp': times,
                            metric_name: values
                        })
                        #df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
                        df['timestamp'] = df['timestamp'].astype(int)
                        df.set_index('timestamp', inplace=True)
                        series_cdf.append(df)
                elif kind == "point_bucket":
                    params = {
                        'query': query.format(int(end-start),int(end-start)),
                        'time': end,
                    }
                    response = requests.get(f'{PROMETHEUS_URL}/api/v1/query', params=params)
                    response.raise_for_status()  # Raise exception if invalid response
                    data = response.json()['data']['result']
                    
                    x_values = []
                    y_values = []

                    for result in data:
                        if 'metric' in result and 'le' in result['metric'] and result['metric']['le'] != '+Inf':
                            x_values.append(float(result['metric']['le']))
                            y_values.append(float(result['value'][1]))
                    df = pd.DataFrame({
                        'timebucket': x_values,
                        metric_name: y_values
                    })
                    df = df.sort_values(by='timebucket')
                    df[metric_name] = df[metric_name] / df[metric_name].max()
                    df.to_csv('data/bench_data{}/{}-bucket-{}-{}.csv'.format(TEST_COUNT,exec,metric_name,bench_params), index=False)

            if len(series_cdf) > 0:
                merged_range_df = pd.concat(series_cdf, axis=1)
                merged_range_df.reset_index(inplace=True)
                merged_range_df.to_csv('data/bench_data{}/{}-series-{}.csv'.format(TEST_COUNT,exec,bench_params), index=False)


def stress_data_deal():
    with open(META_DATA_FILE, 'r') as file:
        content = file.read()
        pattern = r"(start|end) time (\d+\.\d+) file: ./data/(\w+)-(exec\d+)-([\w\-]+)"
        data =  re.findall(pattern, content)
    
    grouped_data = {}

    for entry in data:
        action, time, task_type, exec_name, load = entry
        if exec_name == "exec70":
            break
        if exec_name not in grouped_data:
            grouped_data[exec_name] = {"workload_start": None, "workload_end": None, 
                                       "stress_start": None, "stress_end": None, 
                                       "params": load}
        
        if task_type == "workload" and action == "start":
            grouped_data[exec_name]["workload_start"] = time
        elif task_type == "workload" and action == "end":
            grouped_data[exec_name]["workload_end"] = time
        elif task_type == "stress" and action == "start":
            grouped_data[exec_name]["stress_start"] = time
        elif task_type == "stress" and action == "end":
            grouped_data[exec_name]["stress_end"] = time
    
    result = []
    for exec_name, timings in grouped_data.items():
        exec_name = re.sub(r'(\d+)', lambda x: f"{int(x.group(1)):03}", exec_name)
        result.append((
            timings["workload_start"], timings["workload_end"], 
            timings["stress_start"], timings["stress_end"], 
            timings["params"], exec_name
        ))
    
    with open(META_DATA_OUT_FILE, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["workload_start","workload_end", "stress_start","stress_end","params","exec"])  # Write header
        writer.writerows(result)

def csvs_to_excel():
    all_files = os.listdir('data/bench_data{}'.format(TEST_COUNT))
    csv_files = [file for file in all_files if file.endswith('.csv')]

    series = []
    bucket = []
    for file in csv_files:
        if 'series' in file:
            series.append(file)
        elif 'bucket' in file:
            bucket.append(file)
            
    with pd.ExcelWriter('data/bench_analysis{}/series.xlsx'.format(TEST_COUNT), engine='openpyxl') as writer:
        for file in series:
            df = pd.read_csv('data/bench_data{}/{}'.format(TEST_COUNT,file))
            sheet_name = file.split('-')[0]
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    with pd.ExcelWriter('data/bench_analysis{}/bucket.xlsx'.format(TEST_COUNT), engine='openpyxl') as writer:
        for file in bucket:
            df = pd.read_csv('data/bench_data{}/{}'.format(TEST_COUNT,file))
            sheet_name = file.split('-')[0]
            df.to_excel(writer, sheet_name=sheet_name, index=False)

if __name__ == "__main__":
    # # 根据日志提取数据
    stress_data_deal()

    # # 查询prometheus数据并保存到文件
    fetch_data_save_to_csv()

    csvs_to_excel()

    
